In [16]:
import json

basepath = '/home/xilini/par-gen/01-par-gen/model/scores/'

with open('/home/xilini/par-gen/01-par-gen/data/dataset_paragraphs_v1.json', 'r') as s:
    data = json.load(s)['images']

with open(basepath + 'VISUAL-beam_2_cidertest.json', 'r') as f1:
    VISUAL = json.load(f1)

In [17]:
# construct filepath to the image (based on the Stanford servers)
stanford_path = 'https://cs.stanford.edu/people/rak248/'
image_loc = {}
for item in data:
    if item['split'] == 'test':
        filepath = item['filepath']
        image_id = item['stanford_par_id']
        image_path = stanford_path + str(filepath) + '/' + str(image_id) + '.jpg'
        image_loc[image_id] = image_path
#with open('/home/xilini/par-gen/01-par-gen/data/stanford_loc', 'w') as st:
#    json.dump(image_loc, st)

In [18]:
import pandas as pd

In [11]:
VISUAL = VISUAL[:50]

In [19]:
csv_dict = {}
csv_dict['Image'] = []
csv_dict['Paragraph'] = []


for item in VISUAL:
    image_link = image_loc[item['image_id']]
    hyps = item['hypotheses']
    csv_dict['Image'].append(image_link)
    csv_dict['Paragraph'].append(hyps)
    #break

In [20]:
csv_df = pd.DataFrame(data=csv_dict)

In [21]:
csv_df.to_csv('./VISUAL_df.csv', index=False)

In [22]:
csv_df.shape

(2489, 2)

In [ ]:
results_df = pd.read_csv('./Batch_290265_batch_results.csv')

In [ ]:
results_df.columns

In [ ]:
results_df['Answer.feedback']